### UNSUPERVISED MACHINE LEARNING FOR THE CLASSIFICATION OF ASTROPHYSICAL X-RAY SOURCES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min, silhouette_score, silhouette_samples, calinski_harabasz_score
from sklearn.preprocessing import MinMaxScaler
from astropy import stats
from astropy.io.votable import parse

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
#plt.rcParams['figure.figsize'] = (16, 16)
#plt.style.use('ggplot')

In [ ]:
def votable_to_pandas(votable_file):
    votable = parse(votable_file)
    table = votable.get_first_table().to_table(use_names_over_ids=True)
    return table.to_pandas()

In [ ]:
data = votable_to_pandas("../data/corpus.vot")

#### Exploración

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.columns

#### K-Means

In [ ]:
# features_out = ['name', 'ra', 'dec', 'src_area_b','hard_hm','hard_hs', 'hard_ms', 'var_prob_b', 'var_sigma_b',
#             'var_prob_h', 'var_sigma_h',
#         'var_prob_m', 'var_sigma_m', 'var_prob_s', 'var_sigma_s', 'ks_prob_b', 'ks_prob_h',
#        'ks_prob_m', 'ks_prob_s', 'kp_prob_b', 'kp_prob_h', 'kp_prob_m',
#        'kp_prob_s', 'bb_kt']

features = ['src_area_b', 'hard_hm', 'hard_hs', 'hard_ms','powlaw_gamma', 'var_prob_b', 'var_sigma_b',
            'var_prob_h', 'var_sigma_h',
        'var_prob_m', 'var_sigma_m', 'var_prob_s', 'var_sigma_s', 'ks_prob_b', 'ks_prob_h',
       'ks_prob_m', 'ks_prob_s', 'kp_prob_b', 'kp_prob_h', 'kp_prob_m',
       'kp_prob_s', 'bb_kt']

features_lognorm = ['src_area_b', 'bb_kt', 'var_sigma_b', 'var_sigma_h',                              'var_sigma_m', 'var_sigma_s']

features_norm = ['powlaw_gamma']


X_df_out = data.dropna()
X_df = X_df_out[features]
X = X_df.copy().to_numpy()

In [ ]:
X_df.min()

In [ ]:
def lognorm(X_df, X, name_desc, log):
    """Normalize data. If log = True, it applies
    a log transform before.

    Args:
        X_df (pandas.DataFrame) = data array
        X (np.array) = X_df converted
        name_desc (string) = name of the descriptor
        log (bool) = True if apply log transform before norm
    
    Returns:
        [np.array]: Normalized (or lognorm) array.
    """
       
    col = X_df.columns.get_loc(name_desc)
    X_desc = X_df[name_desc]
    
    if log:
        nonzero = X_desc[X_desc!=0]
        minval = np.min(nonzero)/10

        # print(minval)
        X_desc = X_desc + minval

        x = np.log(X_desc.values)  #returns a numpy array
    else:
        x = X_desc.to_numpy()
    min_max_scaler = MinMaxScaler(feature_range=(0,1))
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))
    X[:,col] = x_scaled.flatten()
    
    return X

In [ ]:
for feature in features_lognorm:
    X = lognorm(X_df, X, feature, True)
           
for feature in features_norm:
    X = lognorm(X_df, X, feature, False)

In [ ]:
X_df = pd.DataFrame(X, columns=X_df.columns).dropna()
X = X_df.to_numpy()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,20))

# Fit the data to the visualizer
visualizer.fit(X_df)
visualizer.show()

In [ ]:
kmeans = KMeans(n_clusters=3).fit(X_df)
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X_df)
visualizer.show()  

In [ ]:
kmeans = KMeans(n_clusters=6).fit(X_df)
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X_df)
visualizer.show()  

In [ ]:
kmeans = KMeans(n_clusters=9).fit(X_df)
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X_df)
visualizer.show()  

In [ ]:
kmeans = KMeans(n_clusters=6).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

# Predicting the clusters
labels = kmeans.predict(X)
# Getting the cluster centers
C = kmeans.cluster_centers_

In [ ]:
#%matplotlib notebook
plt.rcParams['figure.figsize'] = (10, 10)
#plt.style.use('ggplot')

colx = X_df.columns.get_loc("hard_hm")
coly = X_df.columns.get_loc("ks_prob_h")
colz = X_df.columns.get_loc("src_area_b")

#colores=['red','blue', 'green', 'k', 'forestgreen', 'darkviolet', 'darkgoldenrod', 'cyan', 'pink'] # 9
colores=['red', 'forestgreen', 'darkviolet', 'darkgoldenrod', 'darkblue', 'pink'] # 6
#colores=['red', 'forestgreen', 'blue'] # 3
asignar=[]
for row in labels:
    asignar.append(colores[row])

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, colx], X[:, coly], X[:, colz], c=asignar,s=20)
ax.scatter(C[:, colx], C[:, coly], C[:, colz], marker='*', c=colores, s=1000)
ax.set(xlabel = X_df.columns[colx])
ax.set(ylabel = X_df.columns[coly])
ax.set(zlabel = X_df.columns[colz])

In [ ]:
colx = X_df.columns.get_loc("hard_hm")
coly = X_df.columns.get_loc("ks_prob_h")
fig = plt.figure()
plt.scatter(X[:, colx], X[:, coly], c=asignar,s=20)
plt.scatter(C[:, colx], C[:, coly], marker='*', c=colores, s=1000)
plt.xlabel(X_df.columns[colx]);
plt.ylabel(X_df.columns[coly]);

In [ ]:
colx = X_df.columns.get_loc("hard_hs")
coly = X_df.columns.get_loc("kp_prob_h")
fig = plt.figure()
plt.scatter(X[:, colx], X[:, coly], c=asignar,s=20)
plt.scatter(C[:, colx], C[:, coly], marker='*', c=colores, s=1000)
plt.xlabel(X_df.columns[colx]);
plt.ylabel(X_df.columns[coly]);

In [ ]:
colx = X_df.columns.get_loc("hard_hm")
coly = X_df.columns.get_loc("hard_hs")
fig = plt.figure()
plt.scatter(X[:, colx], X[:, coly], c=asignar,s=20)
plt.scatter(C[:, colx], C[:, coly], marker='*', c=colores, s=1000)
plt.xlabel(X_df.columns[colx]);
plt.ylabel(X_df.columns[coly]);

In [ ]:
colx = X_df.columns.get_loc("hard_hm")
coly = X_df.columns.get_loc("src_area_b")
fig = plt.figure()
plt.scatter(X[:, colx], X[:, coly], c=asignar,s=20)
plt.scatter(C[:, colx], C[:, coly], marker='*', c=colores, s=1000)
plt.xlabel(X_df.columns[colx]);
plt.ylabel(X_df.columns[coly]);

In [ ]:
sil_samples = silhouette_samples(X, labels)
prob_sil_samples = (1 + sil_samples)/2

In [ ]:
X_df_out_final = X_df_out.copy()
X_df_out_final['cluster'] = labels
X_df_out_final['silhouette_prob'] = prob_sil_samples
X_df_out_final['silhouette'] = sil_samples

In [ ]:
X_np_out=X_df_out.to_numpy()

colx = X_df_out.columns.get_loc("ra")
coly = X_df_out.columns.get_loc("dec")
fig = plt.figure()
plt.scatter(X_np_out[:, colx], X_np_out[:, coly], c=asignar,s=20)
plt.xlabel(X_df_out.columns[colx]);
plt.ylabel(X_df_out.columns[coly]);
plt.show()

In [ ]:
X_np_out=X_df_out.to_numpy()
colors = sns.color_palette()[0:6]

colx = X_df_out.columns.get_loc("ra")
coly = X_df_out.columns.get_loc("dec")
fig = plt.figure()
s = sns.scatterplot(data=X_df_out, x='ra', y='dec', hue=X_df_out["cluster"], palette=colores, s=40);
s.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, title='cluster')
plt.xlabel(X_df_out.columns[colx]);
plt.ylabel(X_df_out.columns[coly]);
plt.show()

## Probability alternatives:

### Distancias:

In [ ]:
dists = kmeans.transform(X)

In [ ]:
dists

### c-means

In [ ]:
import numpy.matlib

def soft_clustering_weights(data, cluster_centres, **kwargs):
    
    """
    Function to calculate the weights from soft k-means
    data: Array of data. shape = N x F, for N data points and F Features
    cluster_centres: Array of cluster centres. shape = Nc x F, for Nc number of clusters. Input kmeans.cluster_centres_ directly.
    param: m - keyword argument, fuzziness of the clustering. Default 2
    """
    
    # Fuzziness parameter m>=1. Where m=1 => hard segmentation
    m = 2
    if 'm' in kwargs:
        m = kwargs['m']
    
    Nclusters = cluster_centres.shape[0]
    Ndp = data.shape[0]
    Nfeatures = data.shape[1]

    # Get distances from the cluster centres for each data point and each cluster
    EuclidDist = np.zeros((Ndp, Nclusters))
    for i in range(Nclusters):
        EuclidDist[:,i] = np.sum((data-np.matlib.repmat(cluster_centres[i], Ndp, 1))**2,axis=1)
    

    
    # Denominator of the weight from wikipedia:
    invWeight = EuclidDist**(2/(m-1))*np.matlib.repmat(np.sum((1./EuclidDist)**(2/(m-1)),axis=1).reshape(-1,1),1,Nclusters)
    Weight = 1./invWeight
    
    return Weight

In [ ]:
soft_clustering_weights(X, centroids)

In [ ]:
np.max(soft_clustering_weights(X, centroids), axis = 1)

### Silhoutte: 

In [ ]:
sil = []
CH = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i).fit(X_df)
    labels = kmeans.predict(X)

    sil.append(silhouette_score(X, labels, metric = 'euclidean'))
    CH.append(calinski_harabasz_score(X, labels))

In [ ]:
plt.plot(sil, color='blue')

In [ ]:
plt.plot(CH, color='red')

In [ ]:
sil_samples = silhouette_samples(X, labels)

In [ ]:
sil_samples